In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from datasets import load_dataset

# Load once from the hub
ds = load_dataset("XeIaso/switch-screenshots", split="train")

num_shards = 32  # pick what feels right for your size / parallelism
base_path = "s3://xe-zohar-copy/ds/screenshots_sharded"

for i in range(num_shards):
    # shard() gives you a view over 1/N of the rows
    shard = ds.shard(num_shards=num_shards, index=i)

    out_path = f"{base_path}/shard-{i:05d}.parquet"
    print("writing", out_path)

    shard.to_parquet(out_path)


In [ ]:
from datasets import load_dataset
from pixeltable.io import import_huggingface_dataset

data_files = "s3://xe-zohar-copy/ds/screenshots_sharded/*.parquet"

ds_stream = load_dataset(
    "parquet",
    data_files=data_files,
    split="train",
    streaming=True,
)

screenshots = import_huggingface_dataset("screenshots", ds_stream)